In [11]:
import random
import psycopg2
from psycopg2 import sql
import json

# Define templates for HIPAA, BH, and other questions with options
hipaa_templates = [
    {
        "question": "Have you received HIPAA training in the last {timeframe}?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "Are you familiar with the {rule} Rule under HIPAA?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "Do you know the proper procedures for {action} a HIPAA violation?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "How do you ensure the confidentiality of {info_type}?",
        "options": ["Strict access control", "Encryption", "Training staff"],
        "answer": "Strict access control"
    },
    {
        "question": "Can you explain the consequences of a {violation_type} HIPAA violation?",
        "options": ["Fines", "Legal action", "Reputation damage"],
        "answer": "Fines"
    },
]

bh_templates = [
    {
        "question": "How often do you feel {emotion}?",
        "options": ["Rarely", "Sometimes", "Often", "Always"],
        "answer": "Rarely"
    },
    {
        "question": "Do you have trouble {sleep_issue}?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "Have you experienced a loss of {activity} in activities?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "How would you rate your level of {mental_state} over the past two weeks?",
        "options": ["Low", "Medium", "High"],
        "answer": "Low"
    },
    {
        "question": "Do you have any difficulty {concentration_issue}?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
]

other_templates = [
    {
        "question": "What is your current level of {stress_cause} on a scale from 1 to 10?",
        "options": [str(i) for i in range(1, 11)],
        "answer": "5"
    },
    {
        "question": "How often do you {exercise} per week?",
        "options": [str(i) for i in range(0, 8)],
        "answer": "3"
    },
    {
        "question": "Do you have a {diet_type} diet?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "Do you {habit}?",
        "options": ["Yes", "No"],
        "answer": "Yes"
    },
    {
        "question": "How much {fluid} do you drink daily?",
        "options": [str(i) for i in range(0, 11)],
        "answer": "5"
    },
]

# Possible values for placeholders
hipaa_values = {
    "timeframe": ["year", "month", "quarter"],
    "rule": ["Privacy", "Security"],
    "action": ["reporting", "handling"],
    "info_type": ["patient information", "health records"],
    "violation_type": ["minor", "major"]
}

bh_values = {
    "emotion": ["down, depressed, or hopeless", "anxious or nervous"],
    "sleep_issue": ["falling or staying asleep", "waking up too early"],
    "activity": ["interest or pleasure", "motivation"],
    "mental_state": ["anxiety", "stress"],
    "concentration_issue": ["concentrating on tasks", "focusing"]
}

other_values = {
    "stress_cause": ["work stress", "personal stress"],
    "exercise": ["exercise", "work out"],
    "diet_type": ["balanced", "healthy"],
    "habit": ["smoke", "use tobacco products"],
    "fluid": ["water", "fluids"]
}

# Function to generate a single question with options and answer from a template
def generate_question(template, values):
    question = template["question"].format(**{k: random.choice(v) for k, v in values.items()})
    options = template["options"]
    answer = template["answer"]
    return {"question": question, "options": options, "answer": answer}

# Function to generate a large number of questions
def generate_questions(templates, values, count):
    return [generate_question(random.choice(templates), values) for _ in range(count)]

# Generate 10,000 questions for each category
def generate_all_assessments(total_count):
    count_per_category = total_count // 3
    hipaa_qs = generate_questions(hipaa_templates, hipaa_values, count_per_category)
    bh_qs = generate_questions(bh_templates, bh_values, count_per_category)
    other_qs = generate_questions(other_templates, other_values, count_per_category)
    
    return {
        "HIPAA": hipaa_qs,
        "BH": bh_qs,
        "Other": other_qs
    }

# Database connection details
db_config = {
    'dbname': 'asmt',
    'user': 'postgres',
    'password': 'test',
    'host': 'localhost',
    'port': '5432',
}

# Function to connect to PostgreSQL
def connect_db(config):
    try:
        conn = psycopg2.connect(**config)
        print("Connected to the database.")
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Function to insert questions into the database
def insert_questions(conn, questions):
    with conn.cursor() as cursor:
        for category, qs in questions.items():
            for question in qs:
                cursor.execute(
                    sql.SQL("INSERT INTO asmt.questionnaire (questnr_dtl, questnr_typ_ref_id, questnr_sts_ref_id) VALUES (%s, %s, %s)"),
                    (json.dumps(question), 1, 1)  # Assuming questnr_typ_ref_id and questnr_sts_ref_id are constants
                )
        conn.commit()

# Main function to execute the entire process
def main():
    total_count = 10000
    questions = generate_all_assessments(total_count)
    
    conn = connect_db(db_config)
    if conn:
        insert_questions(conn, questions)
        conn.close()
        print(f"{total_count} questions inserted successfully.")

if __name__ == "__main__":
    main()


Connected to the database.
10000 questions inserted successfully.
